# MS-E2122 - Nonlinear Optimization
### Prof. Fabricio Oliveira

## Homework 2 - Problem 2.3

In [ ]:
# Loading the packages we will need
using ForwardDiff      # Automatic differentiation
using LinearAlgebra    # For using norm()
using Test             # For implementing tests

# Shorthand to the functions to compute gradient and hessian. You can use these to complete parts of the code.
# Type \nabla + Tab to obtain ∇. 
∇(f,x) = ForwardDiff.gradient(f, x)
H(f,x) = ForwardDiff.hessian(f, x)

const tol = 1e-4      # unconstrained method convergence tolerance
const tol_ls = 1e-7   # line search convergence tolerance

## Part 1: implementing the line search : Golden Section method

You are first required to implement the Golden section method as the exact line search to be used. Check the lecture notes for Lecture 5 for a detailed explanation of the method. 

Although it was not covered in class, this is a good opportunity for you to see if what you learn in class can help understand a new method that you haven't seen before, like it will probably be the case in your professional life!

### Input parameters:
- $\theta$: line search function
- $a$: initial lower bound
- $b$: initial upper bound

In [ ]:
function golden_ls(θ; a=0.0, b=10.0, l=tol_ls)
    
    α  = 1/MathConstants.φ        # φ = golden ratio. Here α ≈ 0.618
    
    λ  = a + (1-α)*(b - a)        # NOTE: We do not need to index a, b, λ, and μ like in the lecture 5 pseudocode
    μ  = a + α*(b - a)            #       Instead, we can keep reusing and updating the same variables for notational convenience
    
    θμ = θ(a + α*(b - a))         # Use this variable to compute function values Θ(μₖ₊₁) as in the pseudocode of Lecture 5
    θλ = θ(a + (1 - α)*(b - a))   # Use this variable to compute function values Θ(λₖ₊₁) as in the pseudocode of Lecture 5

    # TODO: Implement what should be inside the while loop of Golden Section method. Use the variables defined above.
    while b - a > l
    # Include your code here, updating the values of λ and μ, 
    # testing which update to do (left or right move) and update 
    # the values of a, b, θλ, and θμ accordingly













    end
    
    return (a + b)/2              # Finally, the function returns the center point of the final interval
end

## Test functions

Use the functions below to test your code. The correct results is commented beside it.

*Tip:* if you want to see the value of any expression you can use the macro `@show`.

Example: 
`@show golden_ls(θ1, a, b)`

In [ ]:
# Tests
θ1(x) = 3x^2 -4x + 6           # Optimal value x = 2/3
θ2(x) = exp(x) - 10x^2 - 20x   # Optimal value x = 4.743864 (in [0,10])

@test abs(golden_ls(θ1) - 2/3) <= 1e-4
@test abs(golden_ls(θ2) - 4.743864) <= 1e-4

## Part 2: implementing the Gradient Descent method

We will now implement the gradient descent method. Below we list all the inputs and outputs we considered for reference.

### Inputs
- f: function to minimize (mandatory)
- x_start: starting point
- max_steps: maximum number of iterations
- ϵ: convergence tolerance.

### Outputs
Return the tuple `(x_iter, f(x), k-1)` 
- x_iter: a matrix with n columns and as many rows as iterations with each point visited
- f(x): function value of all points visited
- k-1: number of iterations, discounted the "iteration 0" for which k = 1.

In [ ]:
function gradient(f; x_start=[0,0], max_steps=1000, ϵ=tol)
    
    x = zeros(max_steps, length(x_start))                 # To save the history of iterations
    x = vcat(x_start', x)                                 # Including starting point                           

    for k = 1:max_steps                                   # Main iteration loop
        
        ∇f = ∇(f, x[k,:])                                 # Gradient at iteration k

        if norm(∇f) < ϵ                                   # Stopping condition: norm of the gradient < tolerance     
            
            return (x[1:k,:], f.(x[i,:] for i=1:k), k-1)  # Return iteration points, function values, and number of iterations
        end                                               # (k-1) as k=1 is "iteration 0".      
        
        # TODO: set the Gradient Descent direction (do not forget to normalise d)

        
        θ(λ) = f(x[k,:] + λ*d)      # Define the line search function 
        λ    = golden_ls(θ)         # Call Golden Section method to compute optimal step size λ  

        # TODO: Update the solution x[k+1,:] at this iteration accordingly

    
    end

    return (x, f.(x[i,:] for i=1:max_steps), max_steps)    # Return iteration points, function values, and number of iterations
end

### Testing the gradient descent method

The cell below implement tests to validate your implementation of the gradient method. We use the same functions for the other methods too.

In [ ]:
# Test functions 
f(x) = 0.26*(x[1]^2 + x[2]^2) - 0.48*x[1]*x[2]  # x_opt = (0,0) and f(x_opt) = 0.
g(x) = exp(x[1] + 3*x[2] - 0.1) + exp(x[1] - 3*x[2] - 0.1) + exp(-x[1] - 0.1) # x_opt ≈ (-0.346574, 0.0) and f(x_opt) = 2.55927
                     
# Testing f(x)
x = [7.0, 3.0]                        # Starting point (7.0,3.0)
(xg, fg, kg) = gradient(f, x_start=x)
@test norm(xg[end,:] - [0.0, 0.0]) <= 1e-2
@test abs(fg[end] - 0.0) <= 1e-2
      
# Testing g(x)
x = [-4.0, -2.0]                      # Starting point (-4.0,-2.0)
(xg, fg, kg) = gradient(g, x_start=x)
@test norm(xg[end,:] - [-0.346574, 0.0]) <= 1e-2
@test abs(fg[end] - 2.55927) <= 1e-2

You can recover the elements of the solution by calling the function as above and then printing (or showing with `@show`) each element.

In [ ]:
(xg, fg, kg) = gradient(f, x_start=x)
println("Optimal solution: $(xg[end,:])")
println("Optimal value: $(fg[end])")
println("Total iterations:  $(kg)")

## Part 3: Newton's method

### Inputs
- f: function to minimize (mandatory)
- x_start: starting point
- max_steps: maximum number of iterations
- ϵ: convergence tolerance.

### Outputs
Return the tuple `(x_iter, f(x), k-1)` 
- x_iter: a matrix with n columns and as many rows as iterations with each point visited
- f(x): function value of all points visited
- k-1: number of iterations, discounted the "iteration 0" for which k = 1.

In [ ]:
function newton(f; x_start=[0,0], max_steps=1000, ϵ=tol)
    
    x = zeros(max_steps, length(x_start))   # To save the history of iterations
    x = vcat(x_start', x)                   # Including starting point 
    
    for k = 1:max_steps                     # Main iteration loop
        
        ∇f = ∇(f, x[k,:])                   # Gradient at iteration k
        
        if norm(∇f) < ϵ                                      # Stopping condition: norm of the gradient < tolerance
            return (x[1:k,:], f.(x[i,:] for i = 1:k), k-1)   # Return iteration points, function values, and number of iterations
        end
        
        # TODO: Update the newton direction

        
        θ(λ) = f(x[k,:] + λ*d)     # Define the line search function 
        λ = golden_ls(θ)           # Call Golden Section method to compute optimal step size λ  

        # TODO: Update the solution x[k+1,:] at this iteration accordingly

    end
    
    return (x, f.(x[i,:] for i = 1:max_steps), max_steps)    # Return iteration points, function values, and number of iterations
end

### Testing the Newton's method

In [ ]:
# Test functions (everything is the same as before, expect for the functin newton() call)
f(x) = 0.26*(x[1]^2 + x[2]^2) - 0.48*x[1]*x[2]  # x_opt = (0,0) and f(x_opt) = 0.
g(x) = exp(x[1] + 3*x[2] - 0.1) + exp(x[1] - 3*x[2] - 0.1) + exp(-x[1] - 0.1) # x_opt ≈ (-0.346574, 0.0) and f(x_opt) = 2.55927

# Testing f(x)
x = [7.0, 3.0]                 # Starting point (7.0,3.0)
(xg, fg, kg) = newton(f, x_start=x)
@test norm(xg[end,:] - [0.0, 0.0]) <= 1e-2
@test abs(fg[end] - 0.0) <=1e-2
      
# Testing g(x)
x = [-4.0, -2.0]               # Starting point (-4.0,-2.0)
(xg, fg, kg) = newton(g, x_start=x)
@test norm(xg[end,:] - [-0.346574, 0.0]) <= 1e-2
@test abs(fg[end] - 2.55927) <=1e-2

## Conjugate Gradient

### Inputs
- f: function to minimize (mandatory)
- x_start: starting point
- max_steps: maximum number of iterations
- ϵ: convergence tolerance.

### Outputs
Return the tuple `(x_iter, f(x), k-1)` 
- x_iter: a matrix with n columns and as many rows as iterations with each point visited
- f(x): function value of all points visited
- k-1: number of iterations, discounted the "iteration 0" for which k = 1.

In [ ]:
function conjugate_gradient(f; x_start=[0,0], max_steps=1000, ϵ=tol)
      
    α = 0.0              # Coefficient for Fletcher-Reeves update
    k = 1                # Iteration number  
    n = length(x_start)  # Dimension of x
    d = -∇(f, x_start)   # Initial direction vector

    x = zeros(max_steps, length(x_start))   # To save the history of iterations
    x = vcat(x_start', x)                   # Including starting point 
    
    while k <= max_steps   # Go through max iterations N and return if at optimum 
        
        for j = 1:n        # Go through each element of x. NOTE: We do not need to use y variables. Instead, 
                           # we can use the empty values in the x variable vector 

            θ(λ) = f(x[k,:] + λ*d)   # Define the line search function 
            λ = golden_ls(θ)   # Call Golden Section method to compute optimal step size λ  
            
            # TODO: Update the value of x[k+1,:] accordingly


            # TODO: Compute value of α using the Fletcher-Reeves update formula


            # TODO: Set the direction vector accordingly

            
            k = k + 1   # Update number of iterations for the y values (here we use x vector instead as mentioned earlier)
            
        end
        
        d = -∇(f, x[k,:]) # Setting d to the gradient for the next cycle of iterations 
        
        if norm(d) < ϵ                                      # Stopping condition: norm of the gradient < tolerance
            return (x[1:k,:], f.(x[i,:] for i = 1:k), k-1)  # Return iteration points, function values, and number of iterations
        end
        
    end
    
    return (x, f.(x[i,:] for i = 1:max_steps), max_steps)    # Return iteration points, function values, and number of iterations
    
end

### Testing the conjugate gradient method

In [ ]:
# Test functions (everything is the same as before, expect for the functin conjugate_gradient() call)
f(x) = 0.26*(x[1]^2 + x[2]^2) - 0.48*x[1]*x[2]  # x_opt = (0,0) and f(x_opt) = 0.
g(x) = exp(x[1] + 3*x[2] - 0.1) + exp(x[1] - 3*x[2] - 0.1) + exp(-x[1] - 0.1) # x_opt ≈ (-0.346574, 0.0) and f(x_opt) = 2.55927

# Testing f(x)
x = [7.0, 3.0]                                      # Starting point (7.0,3.0)
(xg, fg, kg) = conjugate_gradient(f, x_start=x)
@test norm(xg[end,:] - [0.0, 0.0]) <= 1e-2
@test abs(fg[end] - 0.0) <=1e-2
      
# Testing g(x)
x = [-4.0, -2.0]                                    # Starting point (-4.0,-2.0) 
(xg, fg, kg) = conjugate_gradient(g, x_start=x)
@test norm(xg[end,:] - [-0.346574, 0.0]) <= 1e-2
@test abs(fg[end] - 2.55927) <=1e-2

## Plotting the convergence trajectories of the methods

If the above implementations were done correctly, the code below should generate a similar picture to those used in the classes. We will be using your implementaion of the methods to calculate the optimal trajectories and plot the three methods side by side.

Tip: You can use this code as a reference to make plots in the future assignments.

In [ ]:
using Plots 
using LaTeXStrings   # For plotting LaTeX code in tags
pyplot()

f(x) = exp(-(x[1]-3)/2) + exp((4x[2] + x[1])/10) + exp((-4x[2] + x[1])/10) # standard function
# Plotting the contours of the function to be optimised
n = 1000
x1 = range(-10, 25, length=n);
x2 = range(-10, 10, length=n);

contour(x1, x2, (x1,x2) -> f([x1,x2]),
        levels = [3.6, 4, 5, 6 , 8, 10, 15, 20, 30], # set which level curves to plot
        xaxis = (L"$x_1$", (-5,15)),                 # L means we are usign LaTeX notation
        yaxis = (L"$x_2$", (-5,5)),
        title = L"$f(x) = e^{-(x_1-3)/2} + e^{(4x_2 + x_1)/10} + e^{(-4x_2 + x_1)/10}$",  
        clims = (0,35),
        contour_labels = true,
        cbar = false,
        aspect_ratio = :equal
        )

# The optimal value of x for standard function:
xopt = [-(5/6)*(-3 + 2*log(2) - 2*log(5)), 0]
fopt = f(xopt)

# Testing f(x)
x = [12.0, -2.0]
(xg,⋅,⋅) = gradient(f, x_start=x) # We use \cdot to discard outputs we do not need
(xn,⋅,⋅) = newton(f, x_start=x)
(xc,⋅,⋅) = conjugate_gradient(f, x_start=x)

plot!( xg[:,1], xg[:,2], label = "Gradient", marker=:circle)
plot!( xn[:,1], xn[:,2], label = "Newton", marker=:circle)
plot!( xc[:,1], xc[:,2], label = "Conj. grad.", marker=:circle)